In [ ]:
!pip install transformers[sentencepiece]

     |████████████████████████████████| 1.5MB 5.8MB/s 
     |████████████████████████████████| 2.9MB 39.4MB/s 
     |████████████████████████████████| 890kB 45.8MB/s 
     |████████████████████████████████| 1.1MB 37.7MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893261 sha256=e9ab2c3f7966939205d8fee05207674c1fff06f5c8013030c942b40139ae05de
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


# Pipeline

In [ ]:
from transformers import pipeline
classifier = pipeline('sentiment-analysis')

In [ ]:
classifier('I am happy to show you the huggingface transformer library')

[{'label': 'POSITIVE', 'score': 0.9997541904449463}]

In [ ]:
classifier('I am not enough happy to be with you')

[{'label': 'NEGATIVE', 'score': 0.9995957612991333}]

In [ ]:
results = classifier(["We are very happy to show you the 🤗 Transformers library.", "We hope you don't hate it."])
for result in results:
  print(result)

{'label': 'Positive', 'score': 0.6683698892593384}
{'label': 'Positive', 'score': 0.9778962135314941}


By default, the model downloaded for this pipeline is called “distilbert-base-uncased-finetuned-sst-2-english”. 

In [ ]:
classifier = pipeline('sentiment-analysis', model="m3hrdadfi/albert-fa-base-v2-sentiment-binary")

In [ ]:
classifier("من خیلی راضی هستم که اینجا هستم")

[{'label': 'Positive', 'score': 0.9951661229133606}]

other pipeline such as language model

In [ ]:
nlp = pipeline("fill-mask")

Some weights of RobertaForMaskedLM were not initialized from the model checkpoint at distilroberta-base and are newly initialized: ['lm_head.decoder.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
from pprint import pprint
pprint(nlp(f"I can be a good {nlp.tokenizer.mask_token} for you"))

[{'score': 0.08873918652534485,
  'sequence': '<s>I can be a good friend for you</s>',
  'token': 1441,
  'token_str': 'Ġfriend'},
 {'score': 0.04992550238966942,
  'sequence': '<s>I can be a good mentor for you</s>',
  'token': 11989,
  'token_str': 'Ġmentor'},
 {'score': 0.03779127448797226,
  'sequence': '<s>I can be a good listener for you</s>',
  'token': 30443,
  'token_str': 'Ġlistener'},
 {'score': 0.03491486981511116,
  'sequence': '<s>I can be a good dad for you</s>',
  'token': 4252,
  'token_str': 'Ġdad'},
 {'score': 0.02199012227356434,
  'sequence': '<s>I can be a good therapist for you</s>',
  'token': 18931,
  'token_str': 'Ġtherapist'}]


or text generation

In [ ]:
text_generator = pipeline("text-generation")
print(text_generator("I love penguins because they are", max_length=50))

Some weights of GPT2Model were not initialized from the model checkpoint at gpt2 and are newly initialized: ['h.0.attn.masked_bias', 'h.1.attn.masked_bias', 'h.2.attn.masked_bias', 'h.3.attn.masked_bias', 'h.4.attn.masked_bias', 'h.5.attn.masked_bias', 'h.6.attn.masked_bias', 'h.7.attn.masked_bias', 'h.8.attn.masked_bias', 'h.9.attn.masked_bias', 'h.10.attn.masked_bias', 'h.11.attn.masked_bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': 'I love penguins because they are small and so friendly, but most of all because they are extremely brave people, and I love penguins because they are the only penguin to ever see a face, look inside their mouths, tell a story.'}]


# Playing Inside Pipeline

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased-finetuned-sst-2-english")
model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased-finetuned-sst-2-english")

In [ ]:
inputs = tokenizer("We are very happy to show you the 🤗 Transformers library.")

In [ ]:
print(inputs)

{'input_ids': [101, 2057, 2024, 2200, 3407, 2000, 2265, 2017, 1996, 100, 19081, 3075, 1012, 102], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}


In [ ]:
encoded = tokenizer.encode("We are very happy to show you the 🤗 Transformers library.")

In [ ]:
tokenizer.decode(encoded)

'[CLS] we are very happy to show you the [UNK] transformers library. [SEP]'

In [ ]:
batch = tokenizer(
    ["We are very happy to show you the 🤗 Transformers library.", "We hope you don't hate it."],
    padding=True,
    truncation=True,
    max_length=512,
    return_tensors="pt"
)

In [ ]:
batch

{'input_ids': tensor([[  101,  2057,  2024,  2200,  3407,  2000,  2265,  2017,  1996,   100,
         19081,  3075,  1012,   102],
        [  101,  2057,  3246,  2017,  2123,  1005,  1056,  5223,  2009,  1012,
           102,     0,     0,     0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0]])}

In [ ]:
outputs = model(input_ids=batch['input_ids'], attention_mask=batch['input_ids'])

In [ ]:
outputs

SequenceClassifierOutput([('logits', tensor([[-4.0833,  4.3364],
                                   [ 0.0818, -0.0418]], grad_fn=<AddmmBackward>))])

In 🤗 Transformers, all outputs are tuples (with only one element potentially).

In [ ]:
import torch.nn.functional as F
predictions = F.softmax(outputs[0], dim=-1)

In [ ]:
print(predictions)

tensor([[2.2043e-04, 9.9978e-01],
        [5.3086e-01, 4.6914e-01]], grad_fn=<SoftmaxBackward>)
